#### 데이터로딩

In [38]:
import numpy as np


In [37]:
data = np.load("./data/cnn_data.npz")

In [13]:
X_train = data['X_train']
y_train = data['y_train']
X_test = data['X_test']
y_test = data['y_test']

In [14]:
X_train.shape, y_train.shape

((478, 224, 224, 3), (478,))

In [15]:
X_test.shape, y_test.shape

((120, 224, 224, 3), (120,))

#### VGG16모델 전이 학습

In [25]:
from tensorflow.keras.applications import VGG16

In [26]:
vgg16_model = VGG16(input_shape=(224,224,3),
                   include_top=False,
                   weights='imagenet')

In [27]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten


In [29]:
#vgg16_model.trainable = False

In [30]:
for layer in vgg16_model.layers :
    if layer.name == 'block5_conv3' :
        layer.trainable=True
    else :
        layer.trainable=False
        

In [31]:
transfer_model = Sequential()
transfer_model.add(vgg16_model)
transfer_model.add(Flatten())
transfer_model.add(Dense(256,activation='relu'))
transfer_model.add(Dense(3,activation='softmax'))

In [32]:
transfer_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               6422784   
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 771       
Total params: 21,138,243
Trainable params: 8,783,363
Non-trainable params: 12,354,880
_________________________________________________________________


In [33]:
transfer_model.compile(loss="sparse_categorical_crossentropy",
                      optimizer="Adam",
                      metrics=['accuracy'])

ERROR! Session/line number was not unique in database. History logging moved to new session 440


In [35]:
transfer_model.fit(X_train,y_train,
                  validation_split=0.2,
                  epochs=15)

Train on 382 samples, validate on 96 samples
Epoch 1/15
382/382 [==============================] - 2s 4ms/sample - loss: 0.0076 - accuracy: 0.9974 - val_loss: 1.3430 - val_accuracy: 0.8021
Epoch 2/15
382/382 [==============================] - 1s 4ms/sample - loss: 0.0077 - accuracy: 0.9974 - val_loss: 1.4249 - val_accuracy: 0.7812
Epoch 3/15
382/382 [==============================] - 2s 4ms/sample - loss: 0.0052 - accuracy: 0.9974 - val_loss: 1.4600 - val_accuracy: 0.7917
Epoch 4/15
382/382 [==============================] - 2s 4ms/sample - loss: 0.0050 - accuracy: 0.9948 - val_loss: 1.4792 - val_accuracy: 0.7917
Epoch 5/15
382/382 [==============================] - 2s 4ms/sample - loss: 0.0072 - accuracy: 0.9948 - val_loss: 1.5068 - val_accuracy: 0.7917
Epoch 6/15
382/382 [==============================] - 2s 4ms/sample - loss: 0.0039 - accuracy: 0.9974 - val_loss: 1.5328 - val_accuracy: 0.7917
Epoch 7/15
382/382 [==============================] - 2s 4ms/sample - loss: 0.0037 - accura

In [36]:
transfer_model.evaluate(X_test,y_test)

120/120 [==============================] - 2s 16ms/sample - loss: 1.5783 - accuracy: 0.7500


[1.5782870610555013, 0.75]